In [22]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

def file_txt_to_sql(og_filename, filename, table):
    chunk_size = 10_000
    og_file = f'/Users/mmunozcampos/Documents/Comp y reg/{og_filename}'
    db_path = f"/Users/mmunozcampos/Documents/Comp y reg/{filename}"
    
    # Nombres de las columnas
    column_names = [
        'codigo_aseguradora','periodo_informacion', 'tipo_registro', 'run_beneficiario',
        'sexo_beneficiario', 'edad_beneficiario', 'tipo_beneficiario', 'rut_prestador',
        'region_prestador', 'prestador_preferente', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'codigo_prestacion', 'pertenencia_codigo',
        'cobertura_financiamiento_prestacion', 'fecha_bonificacion',
        'tipo_prestador', 'tipo_atencion', 'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago', 'bonificacion_restringida',
        'tipo_plan', 'modalidad_intervencion_quirurgica',
        'ley_urgencia_vital', 'codigo_gpp', 'identificacion_ges', 'none'
    ]
    
    # Creamos SQLite engine con extended timeout y pragma settings
    engine = create_engine(f"sqlite:///{db_path}", 
                          connect_args={"timeout": 300, "check_same_thread": False})
    
    # Prevención de problemas de tipeo
    conn = sqlite3.connect(db_path)
    conn.execute("PRAGMA journal_mode = WAL;")  # Write-Ahead Logging for better performance
    conn.execute("PRAGMA synchronous = NORMAL;")  # Less durability but faster performance
    conn.close()
    
    cols_numericas = [
        'codigo_aseguradora','run_beneficiario', 'rut_prestador',
        'region_prestador', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago'
    ]
    # Correr en chunks
    for i, chunk in enumerate(pd.read_csv(
        og_file,
        delimiter='|',
        names=column_names,
        header=None,
        chunksize=chunk_size,
        encoding='latin1'
    )):
        #Nos encargamos de las columnas numericas
        for col in cols_numericas:
            if 'object' in str(chunk[col].dtype):  # Si la columna es string (u objeto en general)
                chunk[col] = chunk[col].str.strip()
                chunk[col] = pd.to_numeric(chunk[col], errors='coerce')  # Better than direct astype(int) to handle errors
    
        #exportamos a sql
        chunk.iloc[:, :-1].to_sql(table, con=engine, if_exists='append', index=False)
        #para testeo
        #if i == 0:
        #    break
        
    engine.dispose()
    return print(f'{filename} fue un exito')

In [ ]:
#file_txt_to_sql("Prestaciones_Bonificadas_2024t1.txt", "prestaciones_t1.db", "tabla_t1")

In [23]:
#file_txt_to_sql("Prestaciones_Bonificadas_2024t2.txt", "prestaciones_t2.db", "tabla_t2")

prestaciones_t2.db fue un exito


In [24]:
#file_txt_to_sql("Prestaciones_Bonificadas_2024t3.txt", "prestaciones_t3.db", "tabla_t3")

prestaciones_t3.db fue un exito


In [26]:
#file_txt_to_sql("Prestaciones_Bonificadas_2024t4.txt", "prestaciones_t4.db", "tabla_t4")

prestaciones_t4.db fue un exito


In [27]:
import sqlite3
import pandas as pd

db_path = "/Users/mmunozcampos/Documents/Comp y reg/prestaciones_t3.db"
conn = sqlite3.connect(db_path)
query = ''' SELECT *
		FROM tabla_t3 
        LIMIT 100 '''
		
pd.read_sql(query, conn)


,codigo_aseguradora,periodo_informacion,tipo_registro,run_beneficiario,sexo_beneficiario,edad_beneficiario,tipo_beneficiario,rut_prestador,region_prestador,prestador_preferente,...,frecuencia,valor_facturado,valor_bonificado,monto_copago,bonificacion_restringida,tipo_plan,modalidad_intervencion_quirurgica,ley_urgencia_vital,codigo_gpp,identificacion_ges
0,63,2024t3,Curativa,2178806,Femenino,76-80,Carga,2610589,2,Si,...,1,18743,8150,10593,No,Cobertura general,No aplicable,No,X,X
1,63,2024t3,Curativa,2178806,Femenino,76-80,Carga,2610589,2,Si,...,1,18743,6356,12387,No,Cobertura general,No aplicable,No,X,X
2,63,2024t3,Curativa,2178806,Femenino,76-80,Carga,2610589,2,Si,...,1,5499,4124,1375,No,Cobertura general,No aplicable,No,X,X
3,63,2024t3,Curativa,2178806,Femenino,76-80,Carga,2610589,2,Si,...,1,5499,825,4674,No,Cobertura general,No aplicable,No,X,X
4,63,2024t3,Curativa,1737009,Femenino,71-75,Carga,2610589,2,Si,...,1,17520,17520,0,No,Cobertura general,No aplicable,No,X,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,63,2024t3,Curativa,1912217,Masculino,46-50,Cotizante,703530,2,No,...,1,1360,1020,340,No,Cobertura general,No aplicable,No,X,X
96,63,2024t3,Curativa,1912217,Masculino,46-50,Cotizante,703530,2,No,...,1,2960,2220,740,No,Cobertura general,No aplicable,No,X,X
97,63,2024t3,Curativa,176423,Femenino,21-25,Carga,2610589,2,Si,...,1,3427,3427,0,No,Cobertura general,No aplicable,No,X,X
98,63,2024t3,Curativa,176423,Femenino,21-25,Carga,2610589,2,Si,...,1,1440,1440,0,No,Cobertura general,No aplicable,No,X,X
